### Write a function to feach data.

In [1]:
import os
import sys
import pandas as pd
import numpy as np
import warnings 
import scipy.stats as scs
import matplotlib.pyplot as plt
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import FunctionTransformer

In [ ]:
# Suppressing warning messages

warnings.filterwarnings('ignore')

In [ ]:
# setting path to folders

parent_dir = os.path.abspath(os.path.join(os.getcwd(), os.pardir))
scripts_dir = parent_dir+"/scripts"
data_dir = parent_dir+"/data"

sys.path.insert(1, scripts_dir)

In [ ]:
# loading script

from data_summarizing_functions import DataSummarizer
from processing_functions import ConditionalSPRT
from dashboard_viz import VizManager
from processing_functions import DataProcessor

sumar = DataSummarizer()
viz = VizManager()
DP = DataProcessor()

In [ ]:
# loading data

original_df = pd.read_csv(data_dir+"/SmartAd_clean_data.csv")
original_df.head()

In [ ]:
alpha = 0.5
beta = 0.10
odd_ratio = 1.01
c_sprt = ConditionalSPRT()
pipe = Pipeline(
    steps=[
        ("use_custome_cleaner", FunctionTransformer(DP.clean_missing)),
        ("transform data", FunctionTransformer(DP.transform_data)),
        ("ab testing", FunctionTransformer(c_sprt.conditionalSPRT, 
            kw_args={"t1": odd_ratio, "alpha": alpha, "beta": beta})),      
        ])
res = pipe.fit_transform(original_df)

In [ ]:
print(c_sprt.get_output(res))

In [ ]:
c_sprt.plot_output(res)